In [84]:
# Import Liabraries
import pandas as pd # Dataframe and Data Import Library
import numpy as np # Linear Algebra Library

# Plotting Library
import matplotlib.pyplot as plt
from sympy.plotting import plot_implicit
from sympy.parsing.sympy_parser import parse_expr

# Dataset Lib
from sklearn import datasets

In [85]:
# Load the dataset
dataset = pd.read_csv('ex2data1.txt')
iris = datasets.load_iris()
# Feature Scaling
x1_Normalizer = max(dataset.iloc[:,0])
x2_Normalizer = max(dataset.iloc[:,1])
dataset.iloc[:,0] = dataset.iloc[:,0]/x1_Normalizer
dataset.iloc[:,1] = dataset.iloc[:,1]/x2_Normalizer
print(np.array(iris['data']))
# X_train = np.array(X_train)

[[ 5.1  3.5  1.4  0.2]
 [ 4.9  3.   1.4  0.2]
 [ 4.7  3.2  1.3  0.2]
 [ 4.6  3.1  1.5  0.2]
 [ 5.   3.6  1.4  0.2]
 [ 5.4  3.9  1.7  0.4]
 [ 4.6  3.4  1.4  0.3]
 [ 5.   3.4  1.5  0.2]
 [ 4.4  2.9  1.4  0.2]
 [ 4.9  3.1  1.5  0.1]
 [ 5.4  3.7  1.5  0.2]
 [ 4.8  3.4  1.6  0.2]
 [ 4.8  3.   1.4  0.1]
 [ 4.3  3.   1.1  0.1]
 [ 5.8  4.   1.2  0.2]
 [ 5.7  4.4  1.5  0.4]
 [ 5.4  3.9  1.3  0.4]
 [ 5.1  3.5  1.4  0.3]
 [ 5.7  3.8  1.7  0.3]
 [ 5.1  3.8  1.5  0.3]
 [ 5.4  3.4  1.7  0.2]
 [ 5.1  3.7  1.5  0.4]
 [ 4.6  3.6  1.   0.2]
 [ 5.1  3.3  1.7  0.5]
 [ 4.8  3.4  1.9  0.2]
 [ 5.   3.   1.6  0.2]
 [ 5.   3.4  1.6  0.4]
 [ 5.2  3.5  1.5  0.2]
 [ 5.2  3.4  1.4  0.2]
 [ 4.7  3.2  1.6  0.2]
 [ 4.8  3.1  1.6  0.2]
 [ 5.4  3.4  1.5  0.4]
 [ 5.2  4.1  1.5  0.1]
 [ 5.5  4.2  1.4  0.2]
 [ 4.9  3.1  1.5  0.1]
 [ 5.   3.2  1.2  0.2]
 [ 5.5  3.5  1.3  0.2]
 [ 4.9  3.1  1.5  0.1]
 [ 4.4  3.   1.3  0.2]
 [ 5.1  3.4  1.5  0.2]
 [ 5.   3.5  1.3  0.3]
 [ 4.5  2.3  1.3  0.3]
 [ 4.4  3.2  1.3  0.2]
 [ 5.   3.5

In [86]:
def featureMap(x1,x2,degree):
    x = np.ones([len(x2), 1])
    for i in range(1,degree+1):
        for j in range(0,i+1):
            x = np.concatenate( (x, ((x1**(i-j))*(x2**j)).reshape(len(x1),1)), axis=1 )
            
    return x

In [87]:
# Create the X and y trainiing sets
X_init = dataset.iloc[:,[0,1]]
y_init = dataset.iloc[:,2]
def create_dataset(X_train, y_train, degree):
#     print('X Data: \n\n', X_train.head())
#     print('\n\n')
#     print('y Data: \n\n', y_train.head())
    
    # Convert into numpy arrays
    X_train = np.array(X_train)
    y_train = np.array(y_train).reshape(len(y_train),1)
    X_train = featureMap(X_train[:,0],X_train[:,1],degree)

    m = len(y_train) # Number of Examples
    print('Training Set: ', X_train)
    
    return [X_train, y_train]

In [88]:
# Plot the Data
def plot(X,y,w):
    # Seperate Classes
    X_postive1 = X[:,1].reshape(99,1)[y == 1].reshape(60,1)
    X_postive2 = X[:,2].reshape(99,1)[y == 1].reshape(60,1)
    X_postive = np.concatenate( (X_postive1, X_postive2 ), axis =1)

    X_negative1 = X[:,1].reshape(99,1)[y == 0].reshape(39,1)
    X_negative2 = X[:,2].reshape(99,1)[y == 0].reshape(39,1)
    X_negative = np.concatenate( (X_negative1, X_negative2 ), axis =1)
    plt.figure
    plt.scatter(X_postive[:,0], X_postive[:,1], marker = 'x', color = 'r', EdgeColor = 'r')
    plt.scatter(X_negative[:,0], X_negative[:,1], marker = 'o', color='y', EdgeColor = 'b')
    plt.xlabel('Exam 1 Scores')
    plt.ylabel('Exam 1 Scores')
#     if w.any():
#         plot_x = [min(X_train[:,1])-0.1,  max(X_train[:,1])+0.1]
#         plot_y = (-1./weights[2])*(weights[1]*plot_x + weights[0]);
#         plt.plot(plot_x, plot_y)
    plt.show()
    
def ezplot(s):
    #Parse doesn't parse = sign so split
    lhs, rhs = s.replace("^","**").split("=")
    eqn_lhs = parse_expr(lhs)
    eqn_rhs = parse_expr(rhs)

    plot_implicit(eqn_lhs-eqn_rhs)

In [89]:
# Cost function with Derivation
def sigmoid(deriv, x):
    if deriv == True:
        return sigmoid(False, x)*(1-sigmoid(False, x))
    return 1.0/(1 + np.exp(-x))

def perceptron_func(deriv,z):
    return np.array(z>=0, dtype=int)

In [90]:
# Impliment the Cross Entropy Log Loss Function 
def costFunction(X,y,initW,current_epoch,stochastic=False,mini_batch_size=10, cost_func = 'sigmoid'):
    m = len(y)
    J = 0
    derivatives = np.zeros([X.shape[1],1])
    g = np.dot(X,initW)  
    
    if (cost_func == 'sigmoid'):
        hypothesis = sigmoid(False,g)
        J = sum (-1*np.dot(y.reshape(m), np.log(hypothesis)) - ( np.dot((1-y.reshape(m)),np.log(1-hypothesis)) )) / m 
        if not current_epoch%1000:
            print('Error at iteration ', current_epoch, ': ', J)
            print('Still Learning - Trying to get better...\n\n')
    elif (cost_func == 'perceptron'):
        hypothesis = perceptron_func(False,g)
        
#     for i in range(0,len(y)):
#         newCost = ( -1*y[i]*math.log(hypothesis[i]) ) - ( (1-y[i])*math.log(1-hypothesis[i]) )
#         J = J + newCost
        
#     J = J/m
#     print( (hypothesis-y).reshape() )
    
#     derivatives = np.sum( np.dot( (hypothesis - y).reshape(y.shape[0]), X ).reshape(2,1), axis = 1).reshape(X_train.shape[1],1) / m

    if stochastic:
        batch_size = 10
    else:
        batch_size = m

    for i in range(0,len(derivatives)):
        for j in range(0,batch_size):
            diffCost = ( hypothesis[j] - y[j] ) * X[j,i]
            derivatives[i] = derivatives[i] + diffCost
        derivatives[i] = derivatives[i]/m

#     print('Derivatives: ', derivatives)
    return derivatives

In [91]:
def predict(x,weights,cost_func):
    X = x # Create a Copy
    predictions = []
    for i in range(0,len(X)):
        x = featureMap(np.array([X[i,0]]),np.array([X[i,1]]),degree)
        g = np.dot(x,weights)  
        if (cost_func == 'sigmoid'):
            prediction = np.round(sigmoid(False,g))[0,0]
        elif (cost_func == 'perceptron'):
            prediction = np.round(perceptron_func(False,g))[0,0]
        predictions.append(prediction)
        #print('The Predicted Class for the input is: ', prediction)
    return np.array(predictions).reshape(len(predictions),1)

def accuracy(X,y,weights,cost_func):
    test_vals = np.concatenate((X[:,1].reshape(len(X[:,1]),1), X[:,2].reshape(len(X[:,2]),1)), axis = 1)
    predictions = predict(test_vals,weights,cost_func)
    print('Training Accuracy: ', np.mean(predictions == y) * 100, ' %')
    
def train(X, y, initW, num_of_epochs, learning_rate, degree, stochastic=False,mini_batch_size=10, cost_func='sigmoid'):
    [X, y] = create_dataset(X, y, degree)
    if not len(initW):
        # Randomly Initialize the weights
        initW = np.random.randn(X.shape[1],1)
    weights = initW
    print('Initial Weights: ', initW, '\n\n')
    for i in range(1,num_of_epochs):
        deriv = costFunction(X,y,weights,i,stochastic,mini_batch_size, cost_func)
        weights = weights - learning_rate*deriv 
#         print('Weights: ',weights)
#     plot(weights)
    accuracy(X,y,weights,cost_func)
#     predictions = np.dot(X,weights)  
#     print('Training Accuracy: ', np.mean(predictions == y) * 100, ' %')
    return [X,y,weights]

In [96]:
degree = 2 # Needs to be set explicitly globally outside the train function
num_of_epochs = 1000
learning_rate = 5.5
initial_Weights = []

# X_init = iris['data']
# x_1 = np.ones([len(X_init), 1])
# X_init = np.concatenate( (x_1, X_init), axis=1 )
# y_init = np.zeros([150,1])
# y_init[0:49] = 1

In [97]:
[X,y,weights] = train(X_init, y_init, initial_Weights, num_of_epochs, learning_rate, degree, True,10,'sigmoid')

Training Set:  [[ 1.          0.30338937  0.44396933  0.09204511  0.13469558  0.19710877]
 [ 1.          0.35909224  0.73735829  0.12894723  0.26477964  0.54369725]
 [ 1.          0.60286378  0.87295484  0.36344473  0.52627285  0.76205015]
 [ 1.          0.79169019  0.76205933  0.62677336  0.6033149   0.58073443]
 [ 1.          0.45161019  0.56960345  0.20395176  0.25723872  0.32444809]
 [ 1.          0.61212036  0.97615026  0.37469134  0.59752145  0.95286934]
 [ 1.          0.75154118  0.47086355  0.56481414  0.35387335  0.22171248]
 [ 1.          0.76230011  0.88420217  0.58110145  0.67402741  0.78181348]
 [ 1.          0.84578415  0.44031194  0.71535083  0.37240886  0.19387461]
 [ 1.          0.96026858  0.38662381  0.92211574  0.3712627   0.14947797]
 [ 1.          0.75143011  0.30953209  0.56464721  0.23259173  0.09581012]
 [ 1.          0.82448983  0.77356529  0.67978348  0.63779671  0.59840325]
 [ 1.          0.694842    0.98836098  0.48280541  0.68675472  0.97685742]
 [ 1.     